  **Logistic Regression:**
        
-  logistic regression prediction are discrete (only specific value or categories are allowed). we can also view the probability scores underlying the model's classfication

    **Types of Logistic Regression:**
    - Binary (Yes/No)
    - Multi class (orange, pineaple, mango, etc)
    - Ordial (Low, Medium, High)

    **Project:** An MNIST handwritten digit  classifier with Logistic Regression using pytorch 
        

### Packages selection
- The first things is to import all the neccesary packages needed for this project

In [3]:
# import the packages
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader

### Settings
- define all the hyperparameters to be used and needs to be tuned to achive a better accuracy
- Load and explore the data

In [6]:
### Settings ###

# Hyperparameters
input_size = 784 # our image are 28 by 28 pixel in size 
# so each pixel requires a node unit so we have all together 784 node units in the input layer
num_classes = 10 # output to predict is from(0,9)
num_epochs = 5 # Number of training iterations (number of complte pass over the entire train data)
batch_size = 100 # mini_batch size to speed up learning
learning_rate = 0.001

# Dataset

transfm = transforms.ToTensor() # transfrom the dataset object to tensors

# MNIST dataset - images and labels
train_data = datasets.MNIST(root='data',
                           train=True,
                           transform=transfm,
                           download=True)

test_data = datasets.MNIST(root='data',
                          train=False,
                          transform=transfm)

# input pipeline
train_loader = DataLoader(dataset=train_data,
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_data,
                        batch_size=batch_size,
                        shuffle=True)


100.1%

Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw


113.5%

Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw


100.4%

Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw


180.4%

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw
Processing...
Done!


### Define the arcitecture of the model such as 

- The number of input layers; which is determined by the features of the data 
- Number of total hidden layers in the model (iterative)
- Number of hidden units in each layers (iterative)
- The output layer node units is determined by the intended outcome to achive

In [7]:
# Model 
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        predict = self.linear(x) # make a predictions with forward propagation
        return predict

### Loss function and optimizer
- define the specific Loss function to use either cross entropy, MSELoss, etc
- define the oprtization algorithm to use either SGD, Adam, RMSprop, Momentum etc

In [8]:
# Loss function and optimizer
model = LogisticRegression(input_size, num_classes)
criterion = nn.CrossEntropyLoss() # criterion is the same as loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #The optimization algorithm of choice here is SGD

### Training  a model requires the following steps
 - Reset all the gradients to zero (0)
 - Make a forward pass (make a prediction)
 - Calculate the loss
 - Perform back propagation
 - Update all the parameters (weight and biases)

In [11]:
## Training the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28)) # Image flanned into 1D tensor (column vector)
        labels = Variable(labels) # label
        
        # Forward -> Backprop -> optimize
        optimizer.zero_grad() # manually zero the gradient buffers
        output = model(images) # make a predition on the test set
        loss = criterion(output,labels) # compute the loss given the predicted label
                                        # and the actual label
        
        loss.backward() # compute the error gradients with back propagation
        optimizer.step() # optimize the model via stochastic gradient descent
        
        if (i + 1) % 100 == 0:
            print("Epoch {}, loss :{}".format(epoch + 1, loss))

Epoch 1, loss :2.1121270656585693
Epoch 1, loss :2.003312110900879
Epoch 1, loss :1.9126746654510498
Epoch 1, loss :1.8258497714996338
Epoch 1, loss :1.7991440296173096
Epoch 1, loss :1.7355778217315674
Epoch 2, loss :1.649955153465271
Epoch 2, loss :1.5857528448104858
Epoch 2, loss :1.5291244983673096
Epoch 2, loss :1.4944807291030884
Epoch 2, loss :1.4568406343460083
Epoch 2, loss :1.4333828687667847
Epoch 3, loss :1.4340980052947998
Epoch 3, loss :1.4033045768737793
Epoch 3, loss :1.3247932195663452
Epoch 3, loss :1.2865005731582642
Epoch 3, loss :1.257014513015747
Epoch 3, loss :1.3432790040969849
Epoch 4, loss :1.1527975797653198
Epoch 4, loss :1.0792630910873413
Epoch 4, loss :1.1181960105895996
Epoch 4, loss :1.2132201194763184
Epoch 4, loss :1.0781160593032837
Epoch 4, loss :1.1612967252731323
Epoch 5, loss :1.0190832614898682
Epoch 5, loss :0.9546176791191101
Epoch 5, loss :1.009333848953247
Epoch 5, loss :1.025702714920044
Epoch 5, loss :0.9217512607574463
Epoch 5, loss :0.89

### Testing 

In [12]:
# Test the model
correct = 0
total = 0
for images,labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print("Acurracy: {}".format(100 * correct / total))

Acurracy: 83
